Este notebook es una modificación del notebook de `h4_modeling` pero cambiando la variable de edad usando la mediana a la media.

## Preparación del entorno

In [ ]:
import os 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.metrics import silhouette_score

GOLD_DATA_PATH = os.path.join("..", "..", "data/gold/")

## Datos

Los datos tendrá como base la tarjeta de datos 4 (`data_card_4_df.csv`), de la que se eliminará la variable edad para añadir la variable de `Edad media` de la tarjeta de datos 1 (`data_card_1_df.csv`).

La mediana es una medida más representativa que la media, al no estar influida por valores extremos. Sin embargo, en este caso, la forma de calcular la mediana en el paso de preprocesamiento, hace que la variable de edad sea una variable categórica. Por esto, se ha decidido utilizar la media y comparar los resultados.

In [ ]:
# cargar los datos
df1 = pd.read_csv(GOLD_DATA_PATH + "data_card_1_df.csv", sep=";", encoding = 'latin')

df4 = pd.read_csv(GOLD_DATA_PATH + "data_card_4_df.csv", sep=";", encoding = 'latin')
df4.drop(columns=["Unnamed: 0"], inplace=True)


In [ ]:
print("Valores de la columna de edad: ",df4['Mediana edad'].unique())
df4.drop(columns=["Mediana edad"], inplace=True)

In [ ]:
df4 = pd.concat([df4, df1['Edad media']], axis=1)
df4.set_index('Provincias', inplace=True)

In [ ]:
df4.head()

## Estandarización de los datos

In [ ]:
# Escalado de los datos a un rango de 0 a 1
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df4)
df4_scaled = pd.DataFrame(scaled_data, columns=df4.columns, index=df4.index)

df4_scaled.head(3)

## Análisis de componentes principales

Para visualizar los datos.

In [ ]:
# Calculo de PCA
from sklearn.decomposition import PCA

estimator = PCA(n_components=3)
X_pca = estimator.fit_transform(scaled_data)

print("Porcentaje de varianza explicado por cada componente:\n", estimator.explained_variance_ratio_)
pd.DataFrame(np.matrix.transpose(estimator.components_), index=df4.columns)

In [ ]:
#Representación 2D
fig, ax = plt.subplots()
ax.scatter(X_pca[:,0], X_pca[:,1], s=50)

# anotación 
for i in range(0, len(X_pca)):
    ax.annotate(df4.iloc[i, :].name, (X_pca[i, 0], X_pca[i, 1]), fontsize=8)

Si se compara este gráfico con el que se obtiene con la mediana, es una versión espejada.

In [ ]:
# metodo ward, minimiza la varianza intra-cluster
link_matrix_avg = linkage(scaled_data, method='ward', metric='euclidean')

plt.figure(figsize=(8, 5))
dendrogram(link_matrix_avg, labels=df4.index)
plt.show()

In [ ]:
# Visualización de diferente número de clusters
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i in range(2, 10):
    clusters = fcluster(link_matrix_avg, t=i, criterion='maxclust')  # Generar i clusters
    scatter = axes[i-2].scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, s=50, cmap='rainbow')
    coef = silhouette_score(df4_scaled, clusters)
    axes[i-2].legend(*scatter.legend_elements(), title="Clusters")
    axes[i-2].set_title(f'{i} Clusters score: {coef:.3f}')

plt.tight_layout()
plt.show()

In [ ]:
# seleccionamos 2 clusters
clusters = fcluster(link_matrix_avg, t=2, criterion='maxclust')

In [ ]:
# visualización del resultado
plt.figure(figsize=(8, 5))
scatter = plt.scatter(X_pca[:,0], X_pca[:,1], s=50, c=clusters, cmap='plasma_r')

# nombres de las provincias 
for i in range(0, len(X_pca)):
    plt.annotate(df4.iloc[i, :].name, (X_pca[i, 0], X_pca[i, 1]), fontsize=8)

handles, labels = scatter.legend_elements()
legend = plt.legend(handles, labels, title="Clusters", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

## Resultados

2 Clusters:

![img](img/Edad_media_2C.png)

3 Clusters:

![img](img/Edad_media_3C.png)

-------------

----------------------------

## Clasificación y clustering

Para verificar los resultados y sacar conclusiones, se realizará una división de las provincias en norte y sur (criterio?) y se etiquetarán los grupos.

In [ ]:
# Provincias en el dataset
df4.index

In [ ]:
norte=['Araba/Álava', 'Asturias', 'Barcelona', 'Bizkaia', 'Burgos', 'Cantabria', 'Coruña, A', 'Gipuzkoa', 'Girona', 'Guadalajara', 'Huesca', 'León', 'Lleida', 'Lugo', 'Navarra', 'Ourense', 'Palencia', 'Pontevedra', 'Rioja, La', 'Salamanca', 'Segovia', 'Soria', 'Tarragona', 'Teruel', 'Valladolid', 'Zamora', 'Zaragoza', 'Ávila']

df4['Grupo'] = [0 if x in norte else 1 for x in df4.index]

In [ ]:
df4.head()

In [ ]:
df4['Grupo'].value_counts()

## Entrenamiento y validación

Se entrenará un modelo de clasificación, se ha elegido Random Forest, y se sacará el valor de importancia de cada variable para volver a clusterizar y comparar los resultados.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X = df4.drop(columns=["Grupo"])
y = df4["Grupo"]

# se divide el dataset en 30-70 para que el modelo no entrene con demasiados datos y evitar el overfitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

rf = RandomForestClassifier(n_estimators=10, random_state=42)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)

In [ ]:
# Evaluación de resultados
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
cm = confusion_matrix(y_test, pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf.classes_)
cm_display.plot()
plt.title("Confussion Matrix")
print(classification_report(y_test, pred))

El modelo muestra buenos resultados durante el entrenamiento. Pero al tener muy pocos datos, se comprobarán los resultados usando validación cruzada del tipo Leave One Out. Este tipo de validación es muy costoso computacionalmente, pero al tener pocos datos, es la mejor opción.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut

scores = cross_val_score(rf, X, y, cv=LeaveOneOut())
print(f"Accuracy: {scores.mean():.2f}")

In [ ]:
print ('Feature Relevances')
print(pd.DataFrame({'Attributes': X_train.columns,
            'Feature importance':rf.feature_importances_}).sort_values('Feature importance', ascending=False))

In [ ]:
selection = df4_scaled[['Producción media', 'Renta']]

link_matrix_avg = linkage(selection, method='ward', metric='euclidean')

plt.figure(figsize=(8, 5))
dendrogram(link_matrix_avg, labels=df4.index)
plt.show()

In [ ]:
clusters = fcluster(link_matrix_avg, t=2, criterion='maxclust')
df4['Cluster'] = clusters

r = df4[['Grupo', 'Cluster']]
r.to_csv('cluster.csv', sep=';')

Resultados del clustering jerárquico después de realizar la selección de características:

![img](img/seleccion.png)